# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [126]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import ks_2samp
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


def spaceforunder(value):
    if isinstance(value, str):
        return value.replace(' ', '_')
    else:
        return value


def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [11]:
dummie_var = []


df = pd.read_csv('previsao_de_renda.csv')
df.drop(columns = ['Unnamed: 0', 'data_ref'], axis = 1, inplace=True)
df = df.dropna()



for col in df.columns:
    if df[col].dtype == 'object':
        dummie_var.append(col)


print(f'{dummie_var} são as dummies ')

df.info()

['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'] são as dummies 
<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_cliente             12427 non-null  int64  
 1   sexo                   12427 non-null  object 
 2   posse_de_veiculo       12427 non-null  bool   
 3   posse_de_imovel        12427 non-null  bool   
 4   qtd_filhos             12427 non-null  int64  
 5   tipo_renda             12427 non-null  object 
 6   educacao               12427 non-null  object 
 7   estado_civil           12427 non-null  object 
 8   tipo_residencia        12427 non-null  object 
 9   idade                  12427 non-null  int64  
 10  tempo_emprego          12427 non-null  float64
 11  qt_pessoas_residencia  12427 non-null  float64
 12  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), i

In [47]:
# Definindo os dataframes a serem utilizados nos modelos:
y = df['renda']
y = pd.DataFrame(y)

X = df.drop(columns = 'renda', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state = 40028922)

# Definindo o Dataframe de teste:

df_train = pd.concat([X_train, y_train], axis = 1)
df_train_inicio = df_train

df_test = pd.concat([X_test, y_test], axis = 1)



1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_cliente             12427 non-null  int64  
 1   sexo                   12427 non-null  object 
 2   posse_de_veiculo       12427 non-null  bool   
 3   posse_de_imovel        12427 non-null  bool   
 4   qtd_filhos             12427 non-null  int64  
 5   tipo_renda             12427 non-null  object 
 6   educacao               12427 non-null  object 
 7   estado_civil           12427 non-null  object 
 8   tipo_residencia        12427 non-null  object 
 9   idade                  12427 non-null  int64  
 10  tempo_emprego          12427 non-null  float64
 11  qt_pessoas_residencia  12427 non-null  float64
 12  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(5)
memory usage: 1.2+ MB


In [51]:


y = df['renda']
y = pd.DataFrame(y)
X = df.drop(columns = ['renda'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=40028922)

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)


Index(['id_cliente', 'sexo', 'posse_de_veiculo', 'posse_de_imovel',
       'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil',
       'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia',
       'renda'],
      dtype='object')

In [55]:
# # Este trecho de código faz a limpeza, e o split das bases de teste e treino...

# # 2: Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o # #  na base de testes. Qual o melhor modelo
# # 
# Vamos rodar uma elastic_net com um L1 = a 0...

colunas = str(df.columns)
colunas = colunas.replace("'", '')
colunas = colunas.replace(',', ' +')
print(colunas)

Index([id_cliente + sexo + posse_de_veiculo + posse_de_imovel +
       qtd_filhos + tipo_renda + educacao + estado_civil +
       tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia +
       renda] +
      dtype=object)


In [58]:
modelo = '''renda ~ id_cliente + C(sexo) + posse_de_veiculo +
       posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) +
       C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego +
       qt_pessoas_residencia'''
md = smf.ols(modelo, data = df_train_inicio)
reg = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 0.00000000001
                        , alpha = .1)
reg.summary()

# Na aula, o método que nos ensinaram a usar foi o elastic_net... veja bem: o ElasticNet NÃO ACEITA l1_wt = 0... pra "contornar" isso
# atribuí a ele um valor ínfimo, o que creio que deve resultar em algo próximo... honestamente, se for descontada nota da atividade por
# isso, não faço questão de corrigir pra aumentar. Curiosamente, na doc mostra que o valor 0 deveria ser aceito, e resultar em uma ridge.
# mas não foi o que aconteceu...


# Anotações da variação do alpha: 0 -> R-adj = .254 - .001  R-adj = .254 - .005 -> R-adj = .254  - .01 -> R-adj = .254  - .1 -> R-adj =.254
# Ou seja... aparentemente, o nosso R-adj não varia conforme a variação do nosso alpha...

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                     123.0
Date:                Mon, 19 Aug 2024   Prob (F-statistic):               0.00
Time:                        10:46:14   Log-Likelihood:                -96771.
No. Observations:                9320   AIC:                         1.936e+05
Df Residuals:                    9294   BIC:                         1.938e+05
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                           -3845.5384   2912.154     -1.321      0.187   -9554.000    1862.923
C(sexo)[T.M]                         6114.8266    182.808     33.449      0.000    5756.482    6473.171
posse_de_veiculo[T.True]               93.8866    177.099      0.530      0.596    -253.266     441.039
posse_de_imovel[T.True]               372.4962    176.116      2.115      0.034      27.271     717.722
C(tipo_renda)[T.Bolsista]           -1920.2013   2969.969     -0.647      0.518   -7741.991    3901.589
C(tipo_renda)[T.Empresário]           735.1120    186.559      3.940      0.000     369.416    1100.808
C(tipo_renda)[T.Pensionista]        -2654.1819   2972.336     -0.893      0.372   -8480.611    3172.248
C(tipo_renda)[T.Servidor público]     -63.6463    279.074     -0.228      0.820    -610.692     483.399
C(educacao)[T.Pós graduação]          606.3908   2015.790      0.301      0.764   -3345.000    4557.782
C(educacao)[T.Secundário]             707.9552    910.423      0.778      0.437   -1076.673    2492.584
C(educacao)[T.Superior completo]     1351.7512    913.589      1.480      0.139    -439.084    3142.586
C(educacao)[T.Superior incompleto]    308.9490    986.102      0.313      0.754   -1624.027    2241.925
C(estado_civil)[T.Separado]           359.4190   1370.637      0.262      0.793   -2327.331    3046.169
C(estado_civil)[T.Solteiro]            82.7215   1339.221      0.062      0.951   -2542.446    2707.889
C(estado_civil)[T.União]             -601.8223    314.995     -1.911      0.056   -1219.282      15.637
C(estado_civil)[T.Viúvo]              440.9299   1426.512      0.309      0.757   -2355.347    3237.207
C(tipo_residencia)[T.Casa]           -569.5456    696.829     -0.817      0.414   -1935.483     796.392
C(tipo_residencia)[T.Com os pais]    -594.1802    776.156     -0.766      0.444   -2115.616     927.255
C(tipo_residencia)[T.Comunitário]    -345.4008   1320.587     -0.262      0.794   -2934.042    2243.240
C(tipo_residencia)[T.Estúdio]         605.6802   1248.095      0.485      0.627   -1840.860    3052.221
C(tipo_residencia)[T.Governamental]   133.9741    841.092      0.159      0.873   -1514.750    1782.698
id_cliente                             -0.0189      0.017     -1.104      0.270      -0.052       0.015
qtd_filhos                           -613.2555   1328.744     -0.462      0.644   -3217.885    1991.374
idade                                  35.9884      9.983      3.605      0.000      16.420      55.557
tempo_emprego                         561.3798     12.922     43.442      0.000     536.049     586.710
qt_pessoas_residencia                 672.7587   1325.894      0.507      0.612   -1926.285    3271.802
===================================================================

Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

[0, 0.001, 0.005, 0.01, 0.05, 0.1]

In [60]:
modelo = '''renda ~ C(data_ref) + id_cliente + C(sexo) + posse_de_veiculo +
       posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) +
       C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego +
       qt_pessoas_residencia'''
reg2 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 1
                        , alpha = .1)
reg2.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                     127.9
Date:                Mon, 19 Aug 2024   Prob (F-statistic):               0.00
Time:                        10:46:25   Log-Likelihood:                -96771.
No. Observations:                9320   AIC:                         1.936e+05
Df Residuals:                    9295   BIC:                         1.938e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                           -3951.5661   2883.657     -1.370      0.171   -9604.166    1701.033
C(sexo)[T.M]                         6116.4062    182.699     33.478      0.000    5758.276    6474.537
posse_de_veiculo[T.True]               93.1762    177.069      0.526      0.599    -253.918     440.270
posse_de_imovel[T.True]               372.4497    176.107      2.115      0.034      27.242     717.658
C(tipo_renda)[T.Bolsista]           -1917.9156   2969.807     -0.646      0.518   -7739.389    3903.557
C(tipo_renda)[T.Empresário]           735.9192    186.524      3.945      0.000     370.292    1101.546
C(tipo_renda)[T.Pensionista]        -2654.2211   2972.187     -0.893      0.372   -8480.359    3171.916
C(tipo_renda)[T.Servidor público]     -64.1923    279.052     -0.230      0.818    -611.195     482.811
C(educacao)[T.Pós graduação]          621.5198   2014.859      0.308      0.758   -3328.046    4571.085
C(educacao)[T.Secundário]             724.1683    908.265      0.797      0.425   -1056.229    2504.566
C(educacao)[T.Superior completo]     1365.3523    912.062      1.497      0.134    -422.489    3153.194
C(educacao)[T.Superior incompleto]    324.3918    984.283      0.330      0.742   -1605.020    2253.803
C(estado_civil)[T.Separado]           360.1957   1370.565      0.263      0.793   -2326.413    3046.805
C(estado_civil)[T.Solteiro]            84.5296   1339.136      0.063      0.950   -2540.471    2709.530
C(estado_civil)[T.União]             -600.1422    314.914     -1.906      0.057   -1217.443      17.158
C(estado_civil)[T.Viúvo]              440.2737   1426.439      0.309      0.758   -2355.859    3236.406
C(tipo_residencia)[T.Casa]           -474.8986    595.472     -0.798      0.425   -1642.153     692.356
C(tipo_residencia)[T.Com os pais]    -500.0848    687.709     -0.727      0.467   -1848.145     847.976
C(tipo_residencia)[T.Comunitário]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]         700.1996   1194.572      0.586      0.558   -1641.424    3041.823
C(tipo_residencia)[T.Governamental]   228.6933    759.097      0.301      0.763   -1259.304    1716.691
id_cliente                             -0.0190      0.017     -1.111      0.267      -0.052       0.015
qtd_filhos                           -613.0503   1328.677     -0.461      0.645   -3217.549    1991.448
idade                                  35.9023      9.977      3.599      0.000      16.346      55.459
tempo_emprego                         561.4114     12.921     43.449      0.000     536.083     586.740
qt_pessoas_residencia                 672.5859   1325.828      0.507      0.612   -1926.327    3271.499
===================================================================

In [ ]:
# Alpha - R-Adj
# 0     - 0.254
# .001  - 0.254
# .005  - 0.254
# .01   - 0.254
# .05   - 0.254
# .1    - 0.254

# Novamente, o alpha não fez variar o R-ajustado. Nem o R, também...

Rode um modelo stepwise. Avalie o R-quadrado
 nabvase de testes. Qual o melhor resultado?

Vamos transformar as variáveis booleanas em float, e as variáveis do tipo object em dummies também...

In [62]:
X_train.columns

Index(['id_cliente', 'sexo', 'posse_de_veiculo', 'posse_de_imovel',
       'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil',
       'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia'],
      dtype='object')

In [64]:
X_train = pd.get_dummies(X_train, columns=dummie_var, drop_first=True)

bool_var = [col for col in X_train.columns if X_train[col].dtype == 'bool']

if bool_var:
    X_train[bool_var] = X_train[bool_var].astype(int)
else:
    print('Uai... Algo está errado...')




# X_train[dummie_var].head()
# X_train = pd.get_dummies(X_train, columns=dummie_var, drop_first=True)
# X_train
# X_train[bool_var] = X_train[bool_var].astype(int)

X_train.info()
X_train

<class 'pandas.core.frame.DataFrame'>
Index: 9320 entries, 14755 to 9801
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id_cliente                     9320 non-null   int64  
 1   posse_de_veiculo               9320 non-null   int32  
 2   posse_de_imovel                9320 non-null   int32  
 3   qtd_filhos                     9320 non-null   int64  
 4   idade                          9320 non-null   int64  
 5   tempo_emprego                  9320 non-null   float64
 6   qt_pessoas_residencia          9320 non-null   float64
 7   sexo_M                         9320 non-null   int32  
 8   tipo_renda_Bolsista            9320 non-null   int32  
 9   tipo_renda_Empresário          9320 non-null   int32  
 10  tipo_renda_Pensionista         9320 non-null   int32  
 11  tipo_renda_Servidor público    9320 non-null   int32  
 12  educacao_Pós graduação         9320 non-null   in

,id_cliente,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
14755,9131,1,1,1,41,11.627397,3.0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
9234,15207,0,1,0,43,18.797260,2.0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1373,2367,0,1,0,37,15.035616,2.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7336,1755,0,0,0,28,3.312329,1.0,0,0,1,...,1,0,1,0,0,1,0,0,0,0
11222,13988,1,1,2,27,5.041096,4.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,15835,0,0,0,52,29.646575,2.0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
7571,13504,1,0,2,31,13.852055,4.0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
6319,2555,1,1,1,47,0.556164,3.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6018,127,1,0,0,50,3.441096,2.0,1,0,0,...,1,0,0,0,0,0,0,1,0,0


In [66]:
X_train_a = X_train.dropna()
X_train_a = X_train_a.astype(float)
result = stepwise_selection(X_train_a, y_train)

print(result)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo_M                         with p-value 6.0023e-262
#############
['tempo_emprego', 'sexo_M']
Add  tipo_renda_Empresário          with p-value 3.1459e-05
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário']
Add  idade                          with p-value 6.14467e-05
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade']
Add  educacao_Superior completo     with p-value 4.91919e-05
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo']
Add  posse_de_imovel                with p-value 0.0300379
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'posse_de_imovel']
Add  estado_civil_Solteiro          with p-value 0.0343881
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'posse_de_imovel', 'estado_civil

In [ ]:
# Vamos agora, então, aplicar um modelo usando o nosso result. Para isso, vamos formatar as variáveis selecionadas para serem usadas
# na nossa regressão.

var_stepwise = str(result)
# var_stepwise = var_stepwise.replace('[', '').replace(']', '').replace("'", '').replace(',', ' +')
var_stepwise

In [70]:
df_train = pd.concat([X_train_a, y_train], axis=1)
df_train = df_train.rename(columns={'educacao_Superior completo': 'educacao_Superior_completo'})
df_train.columns

Index(['id_cliente', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'sexo_M',
       'tipo_renda_Bolsista', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'tipo_renda_Servidor público',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior_completo', 'educacao_Superior incompleto',
       'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_União',
       'estado_civil_Viúvo', 'tipo_residencia_Casa',
       'tipo_residencia_Com os pais', 'tipo_residencia_Comunitário',
       'tipo_residencia_Estúdio', 'tipo_residencia_Governamental', 'renda'],
      dtype='object')

In [72]:
regr_stepwise = smf.ols(f'''renda ~  tempo_emprego + sexo_M + tipo_renda_Empresário + idade + educacao_Superior_completo 
                        + posse_de_imovel + estado_civil_Solteiro''', data = df_train).fit()

regr_stepwise.mse_total
regr_stepwise.summary()


# Obtivemos um R² de 255, no entanto, um R² ajustado basicamente identico, o que seria um ponto positivo para o nosso data_frame...

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                     454.9
Date:                Mon, 19 Aug 2024   Prob (F-statistic):               0.00
Time:                        10:53:04   Log-Likelihood:                -96777.
No. Observations:                9320   AIC:                         1.936e+05
Df Residuals:                    9312   BIC:                         1.936e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                  -2592.8965    408.244     -6.351      0.000   -3393.144   -1792.649
tempo_emprego                561.5349     12.803     43.861      0.000     536.439     586.631
sexo_M                      6198.2733    171.027     36.242      0.000    5863.024    6533.523
tipo_renda_Empresário        735.0933    181.449      4.051      0.000     379.413    1090.774
idade                         36.4786      9.274      3.933      0.000      18.299      54.658
educacao_Superior_completo   677.7881    168.176      4.030      0.000     348.126    1007.450
posse_de_imovel              376.9110    171.368      2.199      0.028      40.993     712.829
estado_civil_Solteiro       -526.7732    248.971     -2.116      0.034   -1014.811     -38.736
==============================================================================
Omnibus:                    13566.013   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8870757.249
Skew:                           8.542   Prob(JB):                         0.00
Kurtosis:                     153.171   Cond. No.                         220.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Por algum motivo, o que eu até estranhei, os nossos R-² basicamente não sofreram alteração em todas as mudanças de modelos... no entanto, nem com a variação dos alphas, nem com nada mais. No entanto, devido à menor discrepância entre R e R-ajustado, eu escolheria o modelo rodado com o stepwise como o melhor... vamos analisar a eficiencia do modelo escolhido na base de testes.

In [104]:
df_test = pd.concat([X_test, y_test], axis = 1)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3107 entries, 2017 to 13001
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id_cliente                     3107 non-null   int64  
 1   posse_de_veiculo               3107 non-null   int32  
 2   posse_de_imovel                3107 non-null   int32  
 3   qtd_filhos                     3107 non-null   int64  
 4   idade                          3107 non-null   int64  
 5   tempo_emprego                  3107 non-null   float64
 6   qt_pessoas_residencia          3107 non-null   float64
 7   sexo_M                         3107 non-null   int32  
 8   tipo_renda_Bolsista            3107 non-null   int32  
 9   tipo_renda_Empresário          3107 non-null   int32  
 10  tipo_renda_Pensionista         3107 non-null   int32  
 11  tipo_renda_Servidor público    3107 non-null   int32  
 12  educacao_Pós graduação         3107 non-null   in

In [122]:
df_test.columns = df_test.columns.str.replace(' ', '_')
df_test.columns
regr_stepwise_test = smf.ols(f'''renda ~  tempo_emprego + sexo_M + tipo_renda_Empresário + idade + educacao_Superior_completo 
                        + posse_de_imovel + estado_civil_Solteiro''', data = df_test).fit()
regr_stepwise_test.summary()


# Tivemos um resultado na base de testes superior ao da base de treino! Suponho que isto é extremamente satisfatório, no entanto,
# Também extremamente incomum... e não consigo levantar hipóteses quanto à motivação pra isso...

# Vejamos se conseguimos melhorar o desempenho da nossa regressão aplicando alguma transformação nos dados, diretamente no patsy...

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.258
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     153.7
Date:                Mon, 19 Aug 2024   Prob (F-statistic):          2.44e-195
Time:                        11:09:16   Log-Likelihood:                -31999.
No. Observations:                3107   AIC:                         6.401e+04
Df Residuals:                    3099   BIC:                         6.406e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                  -3067.8688    654.857     -4.685      0.000   -4351.866   -1783.871
tempo_emprego                537.7452     20.974     25.638      0.000     496.620     578.870
sexo_M                      5787.0172    276.892     20.900      0.000    5244.106    6329.928
tipo_renda_Empresário        958.4680    292.455      3.277      0.001     385.043    1531.893
idade                         53.0971     14.895      3.565      0.000      23.893      82.301
educacao_Superior_completo   609.9227    270.013      2.259      0.024      80.501    1139.345
posse_de_imovel              379.5195    272.675      1.392      0.164    -155.122     914.161
estado_civil_Solteiro       -228.8784    385.962     -0.593      0.553    -985.646     527.889
==============================================================================
Omnibus:                     3828.251   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           815880.037
Skew:                           6.417   Prob(JB):                         0.00
Kurtosis:                      81.343   Cond. No.                         220.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [124]:
# Aplicando log na renda:
df_test.columns = df_test.columns.str.replace(' ', '_')
df_test.columns
regr_stepwise_test = smf.ols(f'''np.log(renda) ~  tempo_emprego + sexo_M + tipo_renda_Empresário + idade + educacao_Superior_completo 
                        + posse_de_imovel + estado_civil_Solteiro''', data = df_test).fit()
regr_stepwise_test.summary()

# Incrível! nossos resultados aumentaram EXPRESSIVAMENTE! Com nosso R² aumentando quase na casa de um décimo percentualmente! Tornando
# Nosso modelo muito mais capaz de prever a variação da nossa variável resposta!

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.339
Method:                 Least Squares   F-statistic:                     228.3
Date:                Mon, 19 Aug 2024   Prob (F-statistic):          2.64e-274
Time:                        11:12:46   Log-Likelihood:                -3394.0
No. Observations:                3107   AIC:                             6804.
Df Residuals:                    3099   BIC:                             6852.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      7.1514      0.066    108.805      0.000       7.023       7.280
tempo_emprego                  0.0625      0.002     29.679      0.000       0.058       0.067
sexo_M                         0.7751      0.028     27.889      0.000       0.721       0.830
tipo_renda_Empresário          0.1678      0.029      5.715      0.000       0.110       0.225
idade                          0.0047      0.001      3.157      0.002       0.002       0.008
educacao_Superior_completo     0.1130      0.027      4.169      0.000       0.060       0.166
posse_de_imovel                0.0955      0.027      3.489      0.000       0.042       0.149
estado_civil_Solteiro         -0.0400      0.039     -1.034      0.301      -0.116       0.036
==============================================================================
Omnibus:                        3.592   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.166   Jarque-Bera (JB):                3.520
Skew:                           0.073   Prob(JB):                        0.172
Kurtosis:                       3.076   Cond. No.                         220.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [128]:
regressor = DecisionTreeRegressor(random_state=40028922)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f'O erro quadrado médio obtido foi de {mse}')

O erro quadrado médio obtido foi de 35837872.94573099


In [144]:
# Relembrando, o nosso melhor modelo (O último, usando as variáveis stepwise e uma transformação em renda) teve um MSE de:
X_test = df_test.drop(columns=['renda'])
y_pred_regrstw = regr_stepwise_test.predict(X_test)
mse = mean_squared_error(y_test, y_pred_regrstw)
mse

103862867.09545115

In [ ]:
O mse da nossa árvore de regressão está UMA ORDEM DE GRANDEZA inteira menor do que o R² do nosso modelo de regressão utilizado na
atividade. Portanto, parece, usando o MSE como única métrica, muito mais adequado...